# Dependencies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2

from keras.preprocessing.image import ImageDataGenerator

# Constanst

In [ ]:
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
BATCH_SIZE = 10
EPOCHS = 10
LEARNING_RATE = 1e-4
N_CHANNELS = 1

In [ ]:
base_dir = './content/DataSet2/Denoising'

gaussian_dir = os.path.join(base_dir, 'Gaussian')
gaussian_train_dir = os.path.join(gaussian_dir, 'Train')
gaussian_test_dir = os.path.join(gaussian_dir, 'Test')
gaussian_val_dir = os.path.join(gaussian_dir, 'Validation')

periodic_dir = os.path.join(base_dir, 'Periodic')
Periodic_train_dir = os.path.join(periodic_dir, 'Train')
Periodic_test_dir = os.path.join(periodic_dir, 'Test')
Periodic_val_dir = os.path.join(periodic_dir, 'Validation')

salt_dir = os.path.join(base_dir, 'Salt')
Salt_train_dir = os.path.join(salt_dir, 'Train')
Salt_test_dir = os.path.join(salt_dir, 'Test')
Salt_val_dir = os.path.join(salt_dir, 'Validation')

In [ ]:
data = pd.read_excel('./content/Labels.xlsx')
data.head()

# Preprocessing

In [ ]:
def read_images(folder_path):
    arr = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            if(N_CHANNELS==1):
                arr.append(cv2.imread(file_path,cv2.IMREAD_GRAYSCALE))
            else:
                arr.append(cv2.imread(file_path))
    return np.array(arr)

In [ ]:
def fourier_transform(image):
    
    if(N_CHANNELS==3):
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    signal = np.fft.fftshift(np.fft.fft2(image))
    # image = 20*(np.log(np.abs(signal)))

    # image = (image / np.max(image))*255
    # image = cv2.cvtColor(np.uint8(np.round(image)), cv2.COLOR_GRAY2BGR)

    return signal

In [ ]:
def inverse_fourier_transform(signal):
    image = np.fft.ifft2(np.fft.ifftshift(signal))

    if(N_CHANNELS==3):
        image = (image / np.max(image))*255
        image = cv2.cvtColor(np.uint8(np.round(image)), cv2.COLOR_GRAY2BGR)

    return image

In [ ]:
# X_train = read_images(os.path.join(Periodic_train_dir, 'With-Noise'))
# y_train = read_images(os.path.join(Periodic_train_dir, 'Without-Noise'))

# X_test = read_images(os.path.join(Periodic_test_dir, 'With-Noise'))
# y_test = read_images(os.path.join(Periodic_test_dir, 'Without-Noise'))

# X_val = read_images(os.path.join(Periodic_val_dir, 'With-Noise'))
# y_val = read_images(os.path.join(Periodic_val_dir, 'Without-Noise'))

# Data Loader

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    vertical_flip = True,
    # preprocessing_function = fourier_transform,
)
test_datagen = ImageDataGenerator(
    rescale = 1./255,
    # preprocessing_function = fourier_transform,
)

In [ ]:
train_gen = train_datagen.flow_from_directory(
    Periodic_train_dir,
    shuffle=False,
    target_size=(IMAGE_HEIGHT,IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    color_mode='grayscale'
)
val_gen = train_datagen.flow_from_directory(
    Periodic_val_dir,
    shuffle=False,
    target_size=(IMAGE_HEIGHT,IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
)
test_gen = test_datagen.flow_from_directory(
    Periodic_test_dir,
    shuffle=False,
    target_size=(IMAGE_HEIGHT,IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    color_mode='grayscale'
)

# Model

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Define callbacks for early stopping, model checkpointing, and learning rate reduction
my_callbacks = [
    EarlyStopping(monitor='val_loss', patience=5),
    ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3)
]

# 1

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, Input
from keras.models import Model

def unet_model():
  inputs = Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS))

  conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
  pool1 = MaxPooling2D((2, 2))(conv1)
  conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool1)
  pool2 = MaxPooling2D((2, 2))(conv2)
  conv3 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool2)
  pool3 = MaxPooling2D((2, 2))(conv3)
  conv4 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool3)
  pool4 = MaxPooling2D((2, 2))(conv4)

  up1 = UpSampling2D((2, 2))(conv4)
  conv5 = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(up1)
  up2 = UpSampling2D((2, 2))(conv5)
  conv6 = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(up2)
  up3 = UpSampling2D((2, 2))(conv6)
  conv7 = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(up3)
  up4 = UpSampling2D((2, 2))(conv7)
  conv8 = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(up4)

  outputs = Conv2D(N_CHANNELS, (1, 1), activation='sigmoid')(conv8)

  return Model(inputs=inputs, outputs=outputs)

model = unet_model()
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
history = 0
with tf.device('/GPU:0'):
    history = model.fit(
        train_gen,
        epochs=5,
        callbacks=my_callbacks,
        validation_data=val_gen,
    )

In [ ]:
noisy_image = ...
fourier_image = fourier_transform(noisy_image)
predicted_fourier_image = model.predict(fourier_image)
denoised_image = inverse_fourier_transform(predicted_fourier_image)

# Fourier

In [ ]:
import tensorflow as tf

class ComplexConv2D(tf.keras.layers.Layer):
  def __init__(self, filters, kernel_size, activation='relu', **kwargs):
    super(ComplexConv2D, self).__init__(**kwargs)
    self.filters = filters
    self.kernel_size = kernel_size
    self.activation = activation
    # Initialize real and imaginary weights separately
    self.real_conv = tf.keras.layers.Conv2D(filters, kernel_size, activation=activation)
    self.imag_conv = tf.keras.layers.Conv2D(filters, kernel_size, activation=activation)

  def call(self, inputs):
    # Separate real and imaginary parts
    real_input, imag_input = tf.split(inputs, axis=-1, num_or_size_splits=2)
    # Apply convolutions separately
    real_output = self.real_conv(real_input)
    imag_output = self.imag_conv(imag_input)
    # Combine real and imaginary parts
    output = tf.complex(real_output, imag_output)
    return output

class ComplexConv2DTranspose(tf.keras.layers.Layer):
  def __init__(self, filters, kernel_size, strides=(2, 2), padding='same', activation='relu', **kwargs):
    super(ComplexConv2DTranspose, self).__init__(**kwargs)
    self.filters = filters
    self.kernel_size = kernel_size
    self.strides = strides
    self.padding = padding
    self.activation = activation
    # Initialize real and imaginary transposed convolutions separately
    self.real_conv_t = tf.keras.layers.Conv2DTranspose(filters, kernel_size, strides, padding, activation=activation)
    self.imag_conv_t = tf.keras.layers.Conv2DTranspose(filters, kernel_size, strides, padding, activation=activation)

  def call(self, inputs):
    # Separate real and imaginary parts
    real_input, imag_input = tf.split(inputs, axis=-1, num_or_size_splits=2)
    # Apply transposed convolutions separately
    real_output = self.real_conv_t(real_input)
    imag_output = self.imag_conv_t(imag_input)
    # Combine real and imaginary parts
    output = tf.complex(real_output, imag_output)
    return output

# 2

In [ ]:
from keras.models import Model
from keras.layers import Input, MaxPooling2D, UpSampling2D

def celu(x, alpha=1.0):
  return alpha * (tf.math.exp(x) - 1)

# Model with complex-valued layers
def unet_model():
  inputs = Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 2))  # Input with 2 channels (real and imaginary)

  # Encoder with complex-valued convolutions
  conv1 = ComplexConv2D(32, (3, 3), activation=celu)(inputs)
  pool1 = MaxPooling2D((2, 2))(conv1)
  conv2 = ComplexConv2D(32, (3, 3), activation=celu)(pool1)
  pool2 = MaxPooling2D((2, 2))(conv2)
  conv3 = ComplexConv2D(32, (3, 3), activation=celu)(pool2)
  pool3 = MaxPooling2D((2, 2))(conv3)
  conv4 = ComplexConv2D(32, (3, 3), activation=celu)(pool3)
  pool4 = MaxPooling2D((2, 2))(conv4)

  # Decoder with complex-valued transposed convolutions
  up1 = UpSampling2D((2, 2))(pool4)
  conv5 = ComplexConv2DTranspose(32, (3, 3), activation=celu)(up1)
  up2 = UpSampling2D((2, 2))(conv5)
  conv6 = ComplexConv2DTranspose(32, (3, 3), activation=celu)(up2)
  up3 = UpSampling2D((2, 2))(conv6)
  conv7 = ComplexConv2DTranspose(32, (3, 3), activation=celu)(up3)
  up4 = UpSampling2D((2, 2))(conv7)
  conv8 = ComplexConv2DTranspose(32, (3, 3), activation=celu)(up4)

  # Output with 2 channels (real and imaginary)
  outputs = ComplexConv2D(2, (1, 1), activation='linear')(conv8)

  return Model(inputs=inputs, outputs=outputs)


model = unet_model()
model.compile(metrics=['accuracy'], optimizer='adam', loss='mse')
model.summary()